In [1]:
import os

In [8]:
jss = []
list_of_dfs = []

for js in os.listdir('data/tweets/'):
    tweets_path = 'data/tweets/' + js

    with open(tweets_path) as dataFile:
        data = dataFile.read()
        obj = data[data.find('{') : data.rfind('}')+1]

    raw_tweets = obj.split('\n}, {\n')

    tweets_json = []

    # Loading first and last raw tweet.
    tweets_json.append(json.loads(raw_tweets[0] + '}'))
    tweets_json.append(json.loads('{' + raw_tweets[-1]))

    # Loading every other tweet.
    for raw_tweet in raw_tweets[1:len(raw_tweets)-1]:
        tweet = json.loads('{' + raw_tweet + '\n}')
        tweets_json.append(tweet)

    tweets = pd.DataFrame()

    # Getting timestamp of the tweet! Here we convert the string into datetime and change its timezone from UTC to BRT. 
    date_format = '%a %b %d %H:%M:%S %z %Y'
    tweets.loc[:,'created_at'] = [datetime.strptime(tweet['created_at'], date_format).astimezone(pytz.timezone('America/Sao_Paulo')) for tweet in tweets_json]

    # Mobile? Desktop? Let's check that!
    tweets.loc[:,'source'] = [tweet['source'] for tweet in tweets_json]

    # Favorited or retweeted? How many times?
    tweets.loc[:,'favorited'] = [tweet['favorited'] for tweet in tweets_json]
    tweets.loc[:,'retweeted'] = [tweet['retweeted'] for tweet in tweets_json]
    tweets.loc[:,'favorite_count'] = pd.Series([tweet['favorite_count'] for tweet in tweets_json]).astype(int)
    tweets.loc[:,'retweet_count'] = pd.Series([tweet['retweet_count'] for tweet in tweets_json]).astype(int)

    # And what I tweeted? 
    tweets.loc[:,'full_text'] = [tweet['full_text'] for tweet in tweets_json]

    # If it is a reply, who he/she is?
    tweets.loc[:,'in_reply_to_screen_name'] = [tweet['in_reply_to_screen_name'] if 'in_reply_to_screen_name' in tweet else np.NaN for tweet in tweets_json]

    # Sorting by created at timestamp
    tweets = tweets.sort_values('created_at').reset_index(drop=True)

    # Just giving it a unique ID
    tweets.loc[:,'tweet_id'] = np.arange(0, tweets.shape[0])

    list_of_dfs = list_of_dfs.append(tweets)
    
tweets = pd.concat(list_of_dfs)